## 1. 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings

from tqdm.notebook import tqdm_notebook

warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 100)

# pickle 불러오기
with open('C:\\Users\\user\\Desktop\\추천시스템\\wine.pkl', 'rb') as f:
    df = pickle.load(f)
    
df.head()

,index,이름,aroma,food,종류,가격,생산지역,품종,도수,음용 온도,당도,산도,바디,타닌
0,171165,"쿠엔타비나스, 로스 옐슨",[],[],레드,"740,000원",스페인(Spain) >리오하(Rioja),템프라니요 (Tempranillo),,16~18 ℃,1,3,3,3
1,171164,"쿠엔타비나스, 알로마도",[],[],레드,"188,000원",스페인(Spain) >리오하(Rioja),"템프라니요 (Tempranillo) 80%, 비우라 (Viura) 10%, 말바시아 (Malvasia) 6%, 피노 누아 (Pinot Noir) 4%",,16~18 ℃,1,3,3,3
2,171163,"월즈 엔드, 스터 잇 업",[],[],레드,"560,000원",미국(U.S.A) >캘리포니아(California) >나파 카운티(Napa County) >나파 밸리(Napa Valley),카베르네 소비뇽 (Cabernet Sauvignon),,16~18 ℃,1,3,3,4
3,171162,"월즈 엔드, 록 스테디",[],[],레드,"240,000원",미국(U.S.A) >캘리포니아(California) >나파 카운티(Napa County) >나파 밸리(Napa Valley),"메를로 (Merlot) 49%, 시라/쉬라즈 (Syrah/Shiraz) 18%, 쁘띠 베르도 (Petit Verdot) 15%, 카베르네 프랑 (Cabernet Franc)...",,16~18 ℃,1,3,4,5
4,171159,"월즈 엔드, 레벨 레벨",[],[],화이트,"190,000원",미국(U.S.A) >캘리포니아(California) >나파 카운티(Napa County) >나파 밸리(Napa Valley),"샤르도네 (Chardonnay) , 리슬링 (Riesling)",,8~10 ℃,1,3,1,1


In [2]:
df.shape

(13603, 14)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13603 entries, 0 to 13602
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   13603 non-null  object
 1   이름      13603 non-null  object
 2   aroma   13603 non-null  object
 3   food    13603 non-null  object
 4   종류      13603 non-null  object
 5   가격      13603 non-null  object
 6   생산지역    13603 non-null  object
 7   품종      13603 non-null  object
 8   도수      13603 non-null  object
 9   음용 온도   13603 non-null  object
 10  당도      13603 non-null  int64 
 11  산도      13603 non-null  int64 
 12  바디      13603 non-null  int64 
 13  타닌      13603 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 1.5+ MB


## 2. 데이터 전처리

- 분석에 쓰일 주요 칼럼
    - aroma, food,이름, 종류, 품종, 당도, 산도, 바디, 타닌
    - 도수는 일단 보류
    
    
    
- 칼럼별 전처리
    1. 품종: vectorizer 적용을 위해 위해 각 와인별 품종 비율에 따라 개수로 변환하여 문자로 변환

        - 한국어만 사용
        - 특수기호 제거
        - %가 붙어 있을 시 반올림(퍼센트*0.1)
        - %가 붙어있지 않을 시 10/품종 개수 (e.g. 품종 3개면 3개/3개/3개)
        
        
           
    2. 당도, 산도, 바디, 타닌
        - 각 와인의 당도, 산도, 바디, 타인의 정도는 0~5까지의 숫자
        - 이때 각 숫자를 "정도"로 바꾸어 최종 한 컬럼으로 나타냄
          ex) [당도:1, 산도:3, 바디:3, 타닌:3]은 [약한당도 보통산도 보통바디 보통타닌]

In [4]:
wine_df = df[['aroma','food', '종류', '품종', '당도', '산도', '바디', '타닌']]

### 2.1 품종 전처리

In [5]:
wine_df[['품종']].head()

,품종
0,템프라니요 (Tempranillo)
1,"템프라니요 (Tempranillo) 80%, 비우라 (Viura) 10%, 말바시아 (Malvasia) 6%, 피노 누아 (Pinot Noir) 4%"
2,카베르네 소비뇽 (Cabernet Sauvignon)
3,"메를로 (Merlot) 49%, 시라/쉬라즈 (Syrah/Shiraz) 18%, 쁘띠 베르도 (Petit Verdot) 15%, 카베르네 프랑 (Cabernet Franc)..."
4,"샤르도네 (Chardonnay) , 리슬링 (Riesling)"


In [6]:
wine_df['품종_result'] = np.nan

In [7]:
# 1. '%' 없는 경우

# 퍼센트 없는것 10/갯수 만큼 곱하기
# 단어와 단어사이 띄어쓰기로 나누기 
import re

no_percent_idx = wine_df[~wine_df['품종'].str.contains('%')].index

def extend_word(s):
    li=s.split(',')
    li=[i.replace(' ','') for i in li ]
    li=[re.sub('[^가-힣]','', i) for i in li]
    result=' '.join((li*(int(10/len(li)))))
    return result

wine_df['품종_result']=wine_df['품종'].loc[no_percent_idx].apply(lambda x: extend_word(x))
wine_df.head()

,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result
0,[],[],레드,템프라니요 (Tempranillo),1,3,3,3,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요
1,[],[],레드,"템프라니요 (Tempranillo) 80%, 비우라 (Viura) 10%, 말바시아 (Malvasia) 6%, 피노 누아 (Pinot Noir) 4%",1,3,3,3,NaN
2,[],[],레드,카베르네 소비뇽 (Cabernet Sauvignon),1,3,3,4,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽
3,[],[],레드,"메를로 (Merlot) 49%, 시라/쉬라즈 (Syrah/Shiraz) 18%, 쁘띠 베르도 (Petit Verdot) 15%, 카베르네 프랑 (Cabernet Franc)...",1,3,4,5,NaN
4,[],[],화이트,"샤르도네 (Chardonnay) , 리슬링 (Riesling)",1,3,1,1,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링


In [8]:
# 2. '%' 있는 경우


percent_idx = wine_df[wine_df['품종'].str.contains('%')].index

wine_df['품종_prep'] = wine_df['품종'].apply(lambda x: x.split(',')).values
pattern = re.compile('[^ 가-힣0-9%,]+')
wine_df['품종_prep'] = [pattern.sub("", doc) for doc in wine_df['품종']]

for idx in tqdm_notebook(percent_idx):
    kind_result = ""
    kind_list = wine_df.loc[idx]['품종_prep'].split(',')

    for kind in kind_list:
        text = "".join(kind.split()[:-1])
        try:
            cnt = int(np.round(int(kind.split()[-1].replace('%', '')) * 0.1))
            kind_result += "".join((text+' ')*cnt)
        except:
            pass

    wine_df['품종_result'].loc[idx] = kind_result
    
wine_df[['품종_result']].head()

  0%|          | 0/9321 [00:00<?, ?it/s]

,품종_result
0,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요
1,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아
2,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽
3,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑
4,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링


### 2-2. 당도, 산도, 바디, 타닌 전처리

In [9]:
wine_df[['당도', '산도', '바디', '타닌']].head()

,당도,산도,바디,타닌
0,1,3,3,3
1,1,3,3,3
2,1,3,3,4
3,1,3,4,5
4,1,3,1,1


In [10]:
def generate_flavor_column(df, flavor_list):
    for flavor in flavor_list:
        df['무'+flavor] = np.nan
        df['아주약한'+flavor] = np.nan
        df['약한'+flavor] = np.nan
        df['보통'+flavor] = np.nan
        df['강한'+flavor] = np.nan
        df['아주강한'+flavor] = np.nan
    
    return df

wine_df = generate_flavor_column(wine_df, ['당도', '산도', '바디', '타닌'])
wine_df.head()

,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result,품종_prep,...,약한바디,보통바디,강한바디,아주강한바디,무타닌,아주약한타닌,약한타닌,보통타닌,강한타닌,아주강한타닌
0,[],[],레드,템프라니요 (Tempranillo),1,3,3,3,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,템프라니요,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],[],레드,"템프라니요 (Tempranillo) 80%, 비우라 (Viura) 10%, 말바시아 (Malvasia) 6%, 피노 누아 (Pinot Noir) 4%",1,3,3,3,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,"템프라니요 80%, 비우라 10%, 말바시아 6%, 피노 누아 4%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],[],레드,카베르네 소비뇽 (Cabernet Sauvignon),1,3,3,4,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,카베르네 소비뇽,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],[],레드,"메를로 (Merlot) 49%, 시라/쉬라즈 (Syrah/Shiraz) 18%, 쁘띠 베르도 (Petit Verdot) 15%, 카베르네 프랑 (Cabernet Franc)...",1,3,4,5,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,"메를로 49%, 시라쉬라즈 18%, 쁘띠 베르도 15%, 카베르네 프랑 13%, 카베르네 소비뇽 5%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],[],화이트,"샤르도네 (Chardonnay) , 리슬링 (Riesling)",1,3,1,1,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,"샤르도네 , 리슬링",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# 당도, 산도, 바디, 타닌의 정도에 따른 칼럼 생성

def generate_detail_flavor(df, flavor_list):
    for flavor in flavor_list: #['당도', '산도', '바디', '타닌']
        for i in tqdm_notebook(range(len(df))):
            if df.loc[i,flavor] == 0:
                df.loc[i,'무'+flavor] = int(1)
            elif df.loc[i,flavor] == 1:
                df.loc[i,'아주약한'+flavor] = int(1)
            elif df.loc[i,flavor] == 2:
                df.loc[i,'약한'+flavor] = int(1)
            elif df.loc[i,flavor] == 3:
                df.loc[i,'보통'+flavor] = int(1)
            elif df.loc[i,flavor] == 4:
                df.loc[i,'강한'+flavor] = int(1)
            else:
                df.loc[i,'아주강한'+flavor] = int(1)
       
    return df

wine_df = generate_detail_flavor(wine_df, ['당도', '산도', '바디', '타닌'])

  0%|          | 0/13603 [00:00<?, ?it/s]

  0%|          | 0/13603 [00:00<?, ?it/s]

  0%|          | 0/13603 [00:00<?, ?it/s]

  0%|          | 0/13603 [00:00<?, ?it/s]

In [12]:
wine_df = wine_df.fillna(0)
wine_df[wine_df.columns[10:]] = wine_df[wine_df.columns[10:]].astype(int)
wine_df.head()

,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result,품종_prep,...,약한바디,보통바디,강한바디,아주강한바디,무타닌,아주약한타닌,약한타닌,보통타닌,강한타닌,아주강한타닌
0,[],[],레드,템프라니요 (Tempranillo),1,3,3,3,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,템프라니요,...,0,1,0,0,0,0,0,1,0,0
1,[],[],레드,"템프라니요 (Tempranillo) 80%, 비우라 (Viura) 10%, 말바시아 (Malvasia) 6%, 피노 누아 (Pinot Noir) 4%",1,3,3,3,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,"템프라니요 80%, 비우라 10%, 말바시아 6%, 피노 누아 4%",...,0,1,0,0,0,0,0,1,0,0
2,[],[],레드,카베르네 소비뇽 (Cabernet Sauvignon),1,3,3,4,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,카베르네 소비뇽,...,0,1,0,0,0,0,0,0,1,0
3,[],[],레드,"메를로 (Merlot) 49%, 시라/쉬라즈 (Syrah/Shiraz) 18%, 쁘띠 베르도 (Petit Verdot) 15%, 카베르네 프랑 (Cabernet Franc)...",1,3,4,5,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,"메를로 49%, 시라쉬라즈 18%, 쁘띠 베르도 15%, 카베르네 프랑 13%, 카베르네 소비뇽 5%",...,0,0,1,0,0,0,0,0,0,1
4,[],[],화이트,"샤르도네 (Chardonnay) , 리슬링 (Riesling)",1,3,1,1,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,"샤르도네 , 리슬링",...,0,0,0,0,0,1,0,0,0,0


In [13]:
# 최종 flavor 칼럼 생성

def generate_flavor_word_list(df, flavor_list):
    flavor_result = []
    
    for i in tqdm_notebook(range(len(df))):
        temp = ""
        for flavor in flavor_list:
            if df.loc[i,'무'+flavor] == 1:
                temp += '무'+flavor+" "
            if df.loc[i,'아주약한'+flavor] == 1:
                temp += '아주약한'+flavor+" "
            if df.loc[i,'약한'+flavor] == 1:
                temp += '약한'+flavor+" "
            if df.loc[i,'보통'+flavor] == 1:
                temp += '보통'+flavor+" "
            if df.loc[i,'강한'+flavor] == 1:
                temp += '강한'+flavor+" "
            if df.loc[i,'아주강한'+flavor] == 1:
                temp += '아주강한'+flavor+" "
        
        flavor_result.append(temp)

    return flavor_result

wine_df['flavor'] = generate_flavor_word_list(wine_df, ['당도', '산도', '바디', '타닌'])


  0%|          | 0/13603 [00:00<?, ?it/s]

In [14]:
wine_df[['flavor']].head()

,flavor
0,아주약한당도 보통산도 보통바디 보통타닌
1,아주약한당도 보통산도 보통바디 보통타닌
2,아주약한당도 보통산도 보통바디 강한타닌
3,아주약한당도 보통산도 강한바디 아주강한타닌
4,아주약한당도 보통산도 아주약한바디 아주약한타닌


In [15]:
wine_result = wine_df[['aroma', 'food', '종류', '품종_result', 'flavor']]
wine_result['이름']=df['이름']
wine_result.head()

,aroma,food,종류,품종_result,flavor,이름
0,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 로스 옐슨"
1,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 알로마도"
2,[],[],레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,아주약한당도 보통산도 보통바디 강한타닌,"월즈 엔드, 스터 잇 업"
3,[],[],레드,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,아주약한당도 보통산도 강한바디 아주강한타닌,"월즈 엔드, 록 스테디"
4,[],[],화이트,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,아주약한당도 보통산도 아주약한바디 아주약한타닌,"월즈 엔드, 레벨 레벨"


##  aroma, food를 통한 유사도 측정

In [16]:
wine_result

,aroma,food,종류,품종_result,flavor,이름
0,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 로스 옐슨"
1,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 알로마도"
2,[],[],레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,아주약한당도 보통산도 보통바디 강한타닌,"월즈 엔드, 스터 잇 업"
3,[],[],레드,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,아주약한당도 보통산도 강한바디 아주강한타닌,"월즈 엔드, 록 스테디"
4,[],[],화이트,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,아주약한당도 보통산도 아주약한바디 아주약한타닌,"월즈 엔드, 레벨 레벨"
...,...,...,...,...,...,...
13598,"[디저트, 상큼한 과일]","[디저트, 치즈]",화이트,세미용 세미용 세미용 세미용 세미용 세미용 세미용 소비뇽블랑 소비뇽블랑,약한당도 강한산도 보통바디 아주약한타닌,"깔베, 프리미에 꼬뜨 드 보르도"
13599,[달콤한 과일],"[육류, 치즈]",레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 메를로 메를로,아주약한당도 강한산도 보통바디 보통타닌,"깔베, 메독"
13600,"[견과류, 꽃]",[치즈],레드,메를로 메를로 메를로 메를로 메를로 메를로 메를로 메를로 메를로 메를로,아주약한당도 보통산도 약한바디 약한타닌,"깔베, 쌩떼밀리옹"
13601,"[달콤한 과일, 허브]","[육류, 치즈]",레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카르메네르 메를로,아주약한당도 강한산도 아주강한바디 강한타닌,알타이르


'aroma'와 'food'를 피처 벡터화한 한 후 그 행렬 데이터 값을 코사인 유사도로 비교해보자   

1. 'aroma'와 'food' 칼럼을 카운트 기반으로 피처 백터화 변환 =>  `CountVectorizer`
    - 카운트 기반 벡터화는 카운트 값이 높을수록 중요한 단어로 인식
    - 단순히 단어의 빈도만 고려

1. aroma 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교 → 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성

## 1. aroma 유사도

In [17]:
# 1. aroma

from sklearn.feature_extraction.text import CountVectorizer

# Countvectorzier를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
wine_result['aroma_literal'] = wine_result['aroma'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=1, ngram_range=(1,2)) 
aroma_mat = count_vect.fit_transform(wine_result['aroma_literal'])
print(aroma_mat.shape)


(13603, 73)


- `min_df`: 토큰의 빈도가 min_df로 지정한 값보다 작은 경우에는 무시
- `ngram_range()`: 토큰의 크기를 결정, (1,2)는 토큰 1개 또는 2개로 구성된 단어

In [18]:
# # 코퍼스로부터 각 단어의 빈도 수를 기록
# print(count_vect.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지 살펴보기
print(count_vect.vocabulary_)

{'달콤한': 26, '과일': 9, '허브': 64, '달콤한 과일': 27, '과일 허브': 17, '상큼한': 37, '과일 상큼한': 13, '상큼한 과일': 38, '디저트': 28, '커피': 56, '과일 디저트': 12, '디저트 커피': 35, '커피 허브': 63, '허브 디저트': 68, '기타': 18, '과일 기타': 11, '기타 허브': 25, '기타 상큼한': 21, '스모크': 39, '과일 스모크': 14, '스모크 상큼한': 44, '디저트 허브': 36, '유제품': 48, '과일 유제품': 15, '유제품 상큼한': 52, '허브 상큼한': 69, '허브 커피': 72, '견과류': 0, '견과류 허브': 8, '견과류 달콤한': 2, '기타 디저트': 20, '스모크 유제품': 45, '스모크 디저트': 43, '견과류 상큼한': 4, '디저트 상큼한': 32, '디저트 달콤한': 31, '허브 스모크': 70, '허브 달콤한': 67, '과일 커피': 16, '허브 기타': 66, '스모크 허브': 47, '스모크 기타': 41, '스모크 커피': 46, '커피 상큼한': 61, '디저트 스모크': 33, '유제품 허브': 55, '커피 달콤한': 59, '견과류 유제품': 6, '허브 유제품': 71, '기타 스모크': 22, '스모크 달콤한': 42, '허브 견과류': 65, '기타 커피': 24, '디저트 기타': 30, '유제품 스모크': 53, '유제품 기타': 49, '디저트 유제품': 34, '유제품 커피': 54, '스모크 견과류': 40, '커피 견과류': 57, '과일 견과류': 10, '커피 유제품': 62, '기타 유제품': 23, '기타 달콤한': 19, '유제품 디저트': 51, '견과류 디저트': 3, '견과류 기타': 1, '커피 디저트': 60, '커피 기타': 58, '견과류 스모크': 5, '유제품 달콤한': 50, '견과류 커피': 7, '디저트 견과류': 29}


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

aroma_sim = cosine_similarity(aroma_mat, aroma_mat)
print(aroma_sim.shape)
print(aroma_sim[3766])

(13603, 13603)
[0.         0.         0.         ... 0.         0.59628479 0.50395263]


In [20]:
# 개별 레코드에 대해 가장 aroma 유사도가 높은 순으로 다른 레코드를 추출할 때,
# aroma_sim에서 값이 높은 순으로 정렬된 비교 대상 행의 위치 인덱스 값을 추출

aroma_sim_sorted_ind = aroma_sim.argsort()[:, ::-1]
print(aroma_sim_sorted_ind[3766]) # 3766번 레코드의 경우

[4231 3015 5194 ... 8896 3814    0]


>의문: 'aroma' 칼럼은 값이 없는 데이터도 있어 유사도 비교에 적절한 것인지 ?

In [21]:
aroma_sim

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.50709255],
       [0.        , 0.        , 0.        , ..., 0.        , 0.50709255,
        1.        ]])

In [22]:
#aroma가 빈값인 데이터 13603개 중 3530개 있음
wine_result[wine_result['aroma_literal']=='']

,aroma,food,종류,품종_result,flavor,이름,aroma_literal
0,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 로스 옐슨",
1,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 알로마도",
2,[],[],레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,아주약한당도 보통산도 보통바디 강한타닌,"월즈 엔드, 스터 잇 업",
3,[],[],레드,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,아주약한당도 보통산도 강한바디 아주강한타닌,"월즈 엔드, 록 스테디",
4,[],[],화이트,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,아주약한당도 보통산도 아주약한바디 아주약한타닌,"월즈 엔드, 레벨 레벨",
...,...,...,...,...,...,...,...
13570,[],[튀김],레드,블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드,보통당도 아주약한산도 약한바디 아주약한타닌,"칼로 로씨, 레드 무스캇",
13571,[],[튀김],화이트,블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드 블랜드,보통당도 약한산도 약한바디 아주약한타닌,"칼로 로씨, 캘리포니아 화이트",
13583,[],[치즈],레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 메를로 메를로 메를로 메를로,아주약한당도 보통산도 보통바디 보통타닌,지네스테 메독,
13586,[],[치즈],레드,카베르네소비뇽 메를로 카베르네소비뇽 메를로 카베르네소비뇽 메를로 카베르네소비뇽 메를로 카베르네소비뇽 메를로,아주약한당도 강한산도 강한바디 강한타닌,샤또 오 바쥬 리베랄,


## 2. Food 유사도

In [23]:
# 2. food

# Countvectorzier를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
wine_result['food_literal'] = wine_result['food'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) 
food_mat = count_vect.fit_transform(wine_result['food_literal'])
print(food_mat.shape)


(13603, 105)


In [24]:
# # 코퍼스로부터 각 단어의 빈도 수를 기록
# print(count_vect.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지 살펴보기
print(count_vect.vocabulary_)

{'육류': 53, '해산물': 91, '해산물 육류': 97, '한식': 83, '샐러드': 26, '육류 한식': 57, '한식 샐러드': 85, '식전주': 32, '식전주 샐러드': 34, '치즈': 68, '양식': 40, '양식 치즈': 48, '양식 육류': 46, '디저트': 14, '샐러드 치즈': 29, '디저트 육류': 19, '육류 치즈': 55, '치즈 샐러드': 71, '육류 샐러드': 54, '한식 육류': 88, '양식 해산물': 51, '튀김': 74, '해산물 튀김': 100, '튀김 샐러드': 76, '튀김 육류': 79, '해산물 샐러드': 94, '해산물 치즈': 99, '과일': 0, '디저트 과일': 15, '과일 육류': 3, '해산물 과일': 92, '과일 치즈': 5, '향신료': 103, '과일 샐러드': 1, '해산물 식전주': 95, '식전주 과일': 33, '디저트 해산물': 24, '양식 튀김': 49, '튀김 해산물': 82, '식전주 육류': 36, '튀김 치즈': 80, '샐러드 향신료': 31, '양식 샐러드': 44, '양식 식전주': 45, '해산물 양식': 96, '양식 향신료': 52, '양식 과일': 41, '디저트 치즈': 21, '식전주 치즈': 37, '치즈 육류': 72, '디저트 식전주': 17, '육류 향신료': 59, '과일 해산물': 8, '해산물 한식': 101, '치즈 향신료': 73, '구이': 9, '구이 육류': 10, '해산물 디저트': 93, '디저트 튀김': 22, '식전주 양식': 35, '디저트 한식': 23, '한식 치즈': 89, '양식 디저트': 43, '디저트 샐러드': 16, '양식 한식': 50, '중식': 60, '중식 치즈': 64, '디저트 양식': 18, '치즈 과일': 69, '식전주 튀김': 38, '중식 샐러드': 62, '해산물 중식': 98, '중식 육류': 63, '샐러드 과일': 27, '디저트 향신료': 25, '샐러드 육류'

In [25]:
food_sim = cosine_similarity(food_mat, food_mat)
print(food_sim.shape)
print(food_sim[3766])

(13603, 13603)
[0.         0.         0.         ... 1.         0.57735027 0.57735027]


In [26]:
food_sim

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.57735027,
        0.57735027],
       [0.        , 0.        , 0.        , ..., 0.57735027, 1.        ,
        0.33333333],
       [0.        , 0.        , 0.        , ..., 0.57735027, 0.33333333,
        1.        ]])

In [27]:
# 개별 레코드에 대해 가장 food 유사도가 높은 순으로 다른 레코드를 추출할 때,
# food_sim에서 값이 높은 순으로 정렬된 비교 대상 행의 위치 인덱스 값을 추출 

food_sim_sorted_ind = food_sim.argsort()[:, ::-1]
print(food_sim_sorted_ind[3766]) # 3766번 레코드의 경우

[ 6801  3867 12953 ...  6954  6953     0]


In [28]:
#food가 빈값인 데이터 13603개 중 4517개 있음
wine_result[wine_result['food_literal']=='']

,aroma,food,종류,품종_result,flavor,이름,aroma_literal,food_literal
0,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 로스 옐슨",,
1,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 알로마도",,
2,[],[],레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,아주약한당도 보통산도 보통바디 강한타닌,"월즈 엔드, 스터 잇 업",,
3,[],[],레드,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,아주약한당도 보통산도 강한바디 아주강한타닌,"월즈 엔드, 록 스테디",,
4,[],[],화이트,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,아주약한당도 보통산도 아주약한바디 아주약한타닌,"월즈 엔드, 레벨 레벨",,
...,...,...,...,...,...,...,...,...
13550,[],[],로제,람브루스코 람브루스코 람브루스코 람브루스코 람브루스코 람브루스코 람브루스코 람브루스코 람브루스코 람브루스코,보통당도 약한산도 보통바디 아주약한타닌,타베르넬로 로사토 프리잔테,,
13551,[],[],화이트,기타품종 기타품종 기타품종 기타품종 기타품종 기타품종 기타품종 기타품종 기타품종 기타품종,보통당도 약한산도 보통바디 아주약한타닌,타베르넬로 비앙코 프리잔테,,
13553,[],[],화이트,,아주약한당도 보통산도 약한바디 아주약한타닌,뀌베 다드리앙 화이트,,
13585,[달콤한 과일],[],레드,카베르네소비뇽 메를로 쁘띠베르도 카베르네소비뇽 메를로 쁘띠베르도 카베르네소비뇽 메를로 쁘띠베르도,아주약한당도 보통산도 보통바디 보통타닌,"노블, 메독",달콤한 과일,


In [29]:
#food나 aroma가 빈값인 데이터 13603개 중 6198개 있음
wine_result[(wine_result['food_literal']=='')|(wine_result['aroma_literal']=='')]

,aroma,food,종류,품종_result,flavor,이름,aroma_literal,food_literal
0,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 로스 옐슨",,
1,[],[],레드,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 알로마도",,
2,[],[],레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,아주약한당도 보통산도 보통바디 강한타닌,"월즈 엔드, 스터 잇 업",,
3,[],[],레드,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,아주약한당도 보통산도 강한바디 아주강한타닌,"월즈 엔드, 록 스테디",,
4,[],[],화이트,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,아주약한당도 보통산도 아주약한바디 아주약한타닌,"월즈 엔드, 레벨 레벨",,
...,...,...,...,...,...,...,...,...
13583,[],[치즈],레드,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 메를로 메를로 메를로 메를로,아주약한당도 보통산도 보통바디 보통타닌,지네스테 메독,,치즈
13585,[달콤한 과일],[],레드,카베르네소비뇽 메를로 쁘띠베르도 카베르네소비뇽 메를로 쁘띠베르도 카베르네소비뇽 메를로 쁘띠베르도,아주약한당도 보통산도 보통바디 보통타닌,"노블, 메독",달콤한 과일,
13586,[],[치즈],레드,카베르네소비뇽 메를로 카베르네소비뇽 메를로 카베르네소비뇽 메를로 카베르네소비뇽 메를로 카베르네소비뇽 메를로,아주약한당도 강한산도 강한바디 강한타닌,샤또 오 바쥬 리베랄,,치즈
13589,[허브],[],화이트,세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용 소비뇽블랑 소비뇽블랑,아주강한당도 보통산도 약한바디 아주약한타닌,지네스떼 소테른,허브,


## 3. 맛 (당도, 산도, 바디, 타닌) 유사도 


In [30]:
# flavor의 코사인 유사도 구하기

count_vect = CountVectorizer(min_df=0, ngram_range=(1,1)) 
flavor_mat = count_vect.fit_transform(wine_result['flavor'])
print(flavor_mat.shape)

(13603, 24)


In [31]:
print(count_vect.vocabulary_)

{'아주약한당도': 16, '보통산도': 10, '보통바디': 9, '보통타닌': 11, '강한타닌': 3, '강한바디': 1, '아주강한타닌': 15, '아주약한바디': 17, '아주약한타닌': 19, '약한타닌': 23, '강한산도': 2, '약한바디': 21, '약한당도': 20, '보통당도': 8, '약한산도': 22, '아주강한바디': 13, '무당도': 4, '무산도': 6, '무바디': 5, '무타닌': 7, '아주강한산도': 14, '아주강한당도': 12, '아주약한산도': 18, '강한당도': 0}


In [32]:
flavor_sim = cosine_similarity(flavor_mat, flavor_mat)
print(flavor_sim.shape)
print(flavor_sim[3766])

(13603, 13603)
[0.75 0.75 0.75 ... 0.5  0.25 0.5 ]


In [33]:
flavor_sim

array([[1.  , 1.  , 0.75, ..., 0.5 , 0.25, 0.5 ],
       [1.  , 1.  , 0.75, ..., 0.5 , 0.25, 0.5 ],
       [0.75, 0.75, 1.  , ..., 0.5 , 0.5 , 0.75],
       ...,
       [0.5 , 0.5 , 0.5 , ..., 1.  , 0.25, 0.5 ],
       [0.25, 0.25, 0.5 , ..., 0.25, 1.  , 0.5 ],
       [0.5 , 0.5 , 0.75, ..., 0.5 , 0.5 , 1.  ]])

## 4. 품종 유사도 

In [34]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,1)) 
kind_mat = count_vect.fit_transform(wine_result['품종_result'])
print(kind_mat.shape)

(13603, 414)


In [35]:
print(count_vect.vocabulary_)

{'템프라니요': 335, '비우라': 193, '말바시아': 104, '카베르네소비뇽': 301, '메를로': 115, '시라쉬라즈': 236, '쁘띠베르도': 202, '카베르네프랑': 302, '샤르도네': 215, '리슬링': 81, '세미용': 221, '뮈스까델': 135, '슈냉블랑': 232, '피노누아': 396, '피노그리': 393, '소비뇽블랑': 225, '산지오베제': 211, '몬테풀치아노': 128, '네로다볼라': 43, '피노그리지오': 394, '프리미티보': 388, '알리아니코': 259, '모스카토': 122, '네비올로': 46, '게뷔르츠트라미너': 9, '칼라독': 307, '쌩쏘': 241, '그르나슈누아': 26, '까리냥': 35, '그르나슈': 24, '뮈스까': 132, '소비뇽블랑뮈스케': 226, '그릴로': 29, '무르베드르': 130, '뮈스카데': 140, '쁘띠만생': 200, '비오니에': 191, '마카베오': 95, '자렐로': 284, '빠레야다': 197, '모나스트렐': 117, '가르나차': 1, '말벡': 111, '뮈스까블랑쁘띠그랑': 137, '뮈스까달레산드리': 134, '프라파토': 381, '피노블랑': 399, '베르멘티노': 162, '글레라': 30, '피아노': 405, '토론테스': 336, '리즐링': 82, '네그로아마로': 40, '말바시아네라': 105, '파모소': 358, '트레비아노': 344, '기타품종': 31, '피노뮈니에': 397, '소비뇽그리': 223, '레포스코': 67, '알리칸테부쉐': 260, '쁘띠시라': 204, '온다라비주리': 280, '진판델': 291, '가메이': 7, '카르메네르': 299, '피노네로': 395, '피노타주': 402, '알바리뇨': 263, '페코리노': 369, '베르딜': 161, '마르셀란': 86, '보발': 168, '블랜드': 184, '그뤼너벨트리너': 23, '바르베라': 147, '

In [36]:
kind_sim = cosine_similarity(kind_mat, kind_mat)
print(kind_sim.shape)
print(kind_sim[3766])

(13603, 13603)
[0. 0. 0. ... 0. 0. 0.]


In [37]:
kind_sim

array([[1.        , 0.98473193, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98473193, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.98787834,
        0.95257934],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.10976426,
        0.27216553],
       [0.        , 0.        , 0.98787834, ..., 0.10976426, 1.        ,
        0.97090655],
       [0.        , 0.        , 0.95257934, ..., 0.27216553, 0.97090655,
        1.        ]])

*유사도 종류*
- 아로마 유사도 aroma_sim
- 음식 유사도 food_sim
- 맛(당도,산도,바디,타닌) 유사도 flavor_sim
- 품종 유사도 kind_sim

## 추천시스템 구축
#### 1. 맛(당도,산도,바디,타닌)과 품종 유사도로 추천 시스템 구축

In [44]:
#[’당도', 산도', ‘바디', 타닌', ‘aroma’]를 모두 고려한 similarity 구하기 (weight는 각각 0.5)
new_sim1 = 0.5 * flavor_sim + 0.5 * kind_sim

print(new_sim1)

[[1.         0.99236596 0.375      ... 0.25       0.125      0.25      ]
 [0.99236596 1.         0.375      ... 0.25       0.125      0.25      ]
 [0.375      0.375      1.         ... 0.25       0.74393917 0.85128967]
 ...
 [0.25       0.25       0.25       ... 1.         0.17988213 0.38608276]
 [0.125      0.125      0.74393917 ... 0.17988213 1.         0.73545327]
 [0.25       0.25       0.85128967 ... 0.38608276 0.73545327 1.        ]]


In [43]:
len(df.이름.unique())
#겹치는 이름 있음

13578

In [45]:
data=wine_df.reset_index()
data['이름']=df['이름']
data=data.rename(columns={'index':'id'})

In [46]:
data

,id,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result,...,강한바디,아주강한바디,무타닌,아주약한타닌,약한타닌,보통타닌,강한타닌,아주강한타닌,flavor,이름
0,0,[],[],레드,템프라니요 (Tempranillo),1,3,3,3,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요,...,0,0,0,0,0,1,0,0,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 로스 옐슨"
1,1,[],[],레드,"템프라니요 (Tempranillo) 80%, 비우라 (Viura) 10%, 말바시아 (Malvasia) 6%, 피노 누아 (Pinot Noir) 4%",1,3,3,3,템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 템프라니요 비우라 말바시아,...,0,0,0,0,0,1,0,0,아주약한당도 보통산도 보통바디 보통타닌,"쿠엔타비나스, 알로마도"
2,2,[],[],레드,카베르네 소비뇽 (Cabernet Sauvignon),1,3,3,4,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽,...,0,0,0,0,0,0,1,0,아주약한당도 보통산도 보통바디 강한타닌,"월즈 엔드, 스터 잇 업"
3,3,[],[],레드,"메를로 (Merlot) 49%, 시라/쉬라즈 (Syrah/Shiraz) 18%, 쁘띠 베르도 (Petit Verdot) 15%, 카베르네 프랑 (Cabernet Franc)...",1,3,4,5,메를로 메를로 메를로 메를로 메를로 시라쉬라즈 시라쉬라즈 쁘띠베르도 쁘띠베르도 카베르네프랑,...,1,0,0,0,0,0,0,1,아주약한당도 보통산도 강한바디 아주강한타닌,"월즈 엔드, 록 스테디"
4,4,[],[],화이트,"샤르도네 (Chardonnay) , 리슬링 (Riesling)",1,3,1,1,샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링 샤르도네 리슬링,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 아주약한바디 아주약한타닌,"월즈 엔드, 레벨 레벨"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13598,13598,"[디저트, 상큼한 과일]","[디저트, 치즈]",화이트,"세미용 (Semillon) 70%, 소비뇽 블랑 (Sauvignon Blanc) 25%, 뮈스까델 (Muscadelle) 5%",2,4,3,1,세미용 세미용 세미용 세미용 세미용 세미용 세미용 소비뇽블랑 소비뇽블랑,...,0,0,0,1,0,0,0,0,약한당도 강한산도 보통바디 아주약한타닌,"깔베, 프리미에 꼬뜨 드 보르도"
13599,13599,[달콤한 과일],"[육류, 치즈]",레드,"카베르네 소비뇽 (Cabernet Sauvignon) 75%, 메를로 (Merlot) 15%, 카베르네 프랑 (Cabernet Franc) 5%, 쁘띠 베르도 (Petit ...",1,4,3,3,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 메를로 메를로,...,0,0,0,0,0,1,0,0,아주약한당도 강한산도 보통바디 보통타닌,"깔베, 메독"
13600,13600,"[견과류, 꽃]",[치즈],레드,"메를로 (Merlot) 95%, 카베르네 프랑 (Cabernet Franc) 5%",1,3,2,2,메를로 메를로 메를로 메를로 메를로 메를로 메를로 메를로 메를로 메를로,...,0,0,0,0,1,0,0,0,아주약한당도 보통산도 약한바디 약한타닌,"깔베, 쌩떼밀리옹"
13601,13601,"[달콤한 과일, 허브]","[육류, 치즈]",레드,"카베르네 소비뇽 (Cabernet Sauvignon) 86%, 카르메네르 (Carmenere) 7%, 메를로 (Merlot) 7%",1,4,5,4,카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카베르네소비뇽 카르메네르 메를로,...,0,1,0,0,0,0,1,0,아주약한당도 강한산도 아주강한바디 강한타닌,알타이르


In [103]:
# 와인 이름을 넣으면 해당 와인과 비슷한 Top 10의 id를 return
def general_recommendation(input_name, new_sim = new_sim1):
    
    # 고유 id로 index 찾기 
    idx = data.index[data['이름'] == input_name].tolist() # Int64Index 형식이라 list로 바꾸어줌
    k=data.iloc[idx[0],3] #품종
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(new_sim[idx[0]]).sort_values(ascending = False)
    
    # 와인의 종류가 같은것만(레드, 화이트..)
    # 유사도 Top 10의 index 추출
    dd=data.loc[score_series.index]
    top_10_indexes=list(dd[dd['종류']==k].iloc[1:11].index)
    
    #top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if top_10_indexes[0] == idx[0]:
        top_10_indexes = list(score_series.iloc[1:12].index)
        top_10_indexes.remove(idx[0])
    
    # 추천 와인 이름을 담기 위한 empty list 생성
    top_10_name = []
    
    # id list
    for i in top_10_indexes:
        name = data.loc[i]['이름']
        top_10_name.append(name)
    
    return top_10_name

# test
print(general_recommendation('쿠엔타비나스, 로스 옐슨'))

['마르께스 데 톨레도, 크리안자', '보데가스 페냐피엘, 미로스 오', '벤타 모랄레스 템프라니요', '파소 아 파소 코세차', '비냐 알발리 리제르바', '아르수아가 라 쁠란따', '아르수아가 크리안자', '아임 유어 올가닉 레드', '파이니스트 비냐 델 쿠라 그랑 레쎄르바', '세뇨리오 드 이니에스타, 템프라니오']


#### 시연

In [104]:
#input data
data.iloc[10:11,:]

,id,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result,...,강한바디,아주강한바디,무타닌,아주약한타닌,약한타닌,보통타닌,강한타닌,아주강한타닌,flavor,이름
10,10,[],[],화이트,샤르도네 (Chardonnay),1,3,3,2,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,0,1,0,0,0,아주약한당도 보통산도 보통바디 약한타닌,"도멘 앙드레 무앙전, 퓰리니몽라쉐 르 트레진"


In [105]:
#outputs
re=general_recommendation('도멘 앙드레 무앙전, 퓰리니몽라쉐 르 트레진')
re

['피터 르만 와일드카드 샤도네',
 '얄룸바, 옥스포드 랜딩 샤도네이',
 '하디스, 리미티드 셀러 릴리즈 샤도네이',
 '쟝 부스케 샤도네',
 '시에라 바투코 리제르바 샤도네이',
 '브라운 브라더스, 패트리샤 샤르도네',
 '바라문디, 샤르도네 비오니에',
 '얄리, 레세르바 샤르도네',
 '카스텔로 디 아마, 알 포지오 샤도네이',
 '디 지오르지오, 스테리타 샤도네']

In [106]:
cnt=0
for i in re:
    if cnt==0:
        re_df=data[data['이름']==i]
        cnt=1
    else:
        re_df=pd.concat([re_df,data[data['이름']==i]],axis=0)
    
re_df

,id,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result,...,강한바디,아주강한바디,무타닌,아주약한타닌,약한타닌,보통타닌,강한타닌,아주강한타닌,flavor,이름
10859,10859,"[달콤한 과일, 디저트, 상큼한 과일]","[양식, 육류, 샐러드]",화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,피터 르만 와일드카드 샤도네
6040,6040,"[달콤한 과일, 기타, 상큼한 과일]",[],화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,"얄룸바, 옥스포드 랜딩 샤도네이"
4616,4616,"[허브, 상큼한 과일]","[해산물, 육류, 샐러드]",화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,"하디스, 리미티드 셀러 릴리즈 샤도네이"
13073,13073,[달콤한 과일],[해산물],화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,쟝 부스케 샤도네
306,306,"[달콤한 과일, 기타, 상큼한 과일]","[해산물, 치즈]",화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,시에라 바투코 리제르바 샤도네이
13357,13357,"[견과류, 달콤한 과일, , 상큼한 과일]","[해산물, 샐러드, 치즈]",화이트,샤르도네 (Chardonnay),1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,"브라운 브라더스, 패트리샤 샤르도네"
3051,3051,"[, 달콤한 과일, 허브, 기타, 상큼한 과일]","[해산물, 과일, 육류]",화이트,"샤르도네 (Chardonnay) 95%, 비오니에 (Viognier) 5%",1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,"바라문디, 샤르도네 비오니에"
7012,7012,"[달콤한 과일, 상큼한 과일]","[양식, 해산물, 치즈]",화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,"얄리, 레세르바 샤르도네"
13369,13369,"[견과류, 달콤한 과일, 꽃]","[양식, 샐러드]",화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,"카스텔로 디 아마, 알 포지오 샤도네이"
12666,12666,"[달콤한 과일, 빵]","[양식, 해산물, 육류]",화이트,샤르도네 (Chardonnay) 100%,1,3,3,1,샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네 샤르도네,...,0,0,0,1,0,0,0,0,아주약한당도 보통산도 보통바디 아주약한타닌,"디 지오르지오, 스테리타 샤도네"


#### 2. 맛(당도,산도,바디,타닌), 품종. food 유사도로 추천 시스템 구축
*유사도 종류*
- 음식 유사도 food_sim
- 맛(당도,산도,바디,타닌) 유사도 flavor_sim
- 품종 유사도 kind_sim

In [112]:
new_sim2=0.3 * flavor_sim + 0.2 * kind_sim+ 0.5 * food_sim
new_sim2

array([[0.5       , 0.49694639, 0.225     , ..., 0.15      , 0.075     ,
        0.15      ],
       [0.49694639, 0.5       , 0.225     , ..., 0.15      , 0.075     ,
        0.15      ],
       [0.225     , 0.225     , 0.5       , ..., 0.15      , 0.34757567,
        0.41551587],
       ...,
       [0.15      , 0.15      , 0.15      , ..., 1.        , 0.38562799,
        0.49310824],
       [0.075     , 0.075     , 0.34757567, ..., 0.38562799, 1.        ,
        0.51084798],
       [0.15      , 0.15      , 0.41551587, ..., 0.49310824, 0.51084798,
        1.        ]])

In [125]:
# 와인 이름을 넣으면 해당 와인과 비슷한 Top 10의 id를 return
def food_recommendation(input_name, new_sim = new_sim2):
    
    # 고유 id로 index 찾기 
    idx = data.index[data['이름'] == input_name].tolist() # Int64Index 형식이라 list로 바꾸어줌
    k=data.iloc[idx[0],3] #품종
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(new_sim[idx[0]]).sort_values(ascending = False)
    
    #와인의 종류가 같은것만(레드, 화이트..)
    # 유사도 Top 10의 index 추출
    dd=data.loc[score_series.index]
    top_10_indexes=list(dd[dd['종류']==k].iloc[1:11].index)
    
    #top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if top_10_indexes[0] == idx[0]:
        top_10_indexes = list(score_series.iloc[1:12].index)
        top_10_indexes.remove(idx[0])
    
    # 추천 와인 이름을 담기 위한 empty list 생성
    top_10_name = []
    
    # id list
    for i in top_10_indexes:
        name = data.loc[i]['이름']
        top_10_name.append(name)
    
    return top_10_name

# test
print(food_recommendation('코노 수르, 비씨클레타 샤르도네'))


['블랙 스탈리온 샤르도네', '팬골린, 샤도네', 'G7 레세르바 샤르도네', '폭스 크릭 샤도네', '루이스 펠리페 에드워즈 샤도네이', '글래스 마운틴 샤르도네', '올리비에 르플레브, 샤사뉴 몽라쉐', '그랜트 버지, gb32 샤도네이', '로버트 홀, 샤도네이', '제이콥스 크릭, 리저브 샤르도네']


In [126]:
data[data['이름']=='깔베, 프리미에 꼬뜨 드 보르도']

,id,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result,...,강한바디,아주강한바디,무타닌,아주약한타닌,약한타닌,보통타닌,강한타닌,아주강한타닌,flavor,이름
13598,13598,"[디저트, 상큼한 과일]","[디저트, 치즈]",화이트,"세미용 (Semillon) 70%, 소비뇽 블랑 (Sauvignon Blanc) 25%, 뮈스까델 (Muscadelle) 5%",2,4,3,1,세미용 세미용 세미용 세미용 세미용 세미용 세미용 소비뇽블랑 소비뇽블랑,...,0,0,0,1,0,0,0,0,약한당도 강한산도 보통바디 아주약한타닌,"깔베, 프리미에 꼬뜨 드 보르도"


In [128]:
cnt=0
#outputs
re=food_recommendation('깔베, 프리미에 꼬뜨 드 보르도')
for i in re:
    if cnt==0:
        re_df=data[data['이름']==i]
        cnt=1
    else:
        re_df=pd.concat([re_df,data[data['이름']==i]],axis=0)
    
re_df

,id,aroma,food,종류,품종,당도,산도,바디,타닌,품종_result,...,강한바디,아주강한바디,무타닌,아주약한타닌,약한타닌,보통타닌,강한타닌,아주강한타닌,flavor,이름
7735,7735,"[, 달콤한 과일, 꽃, 견과류, 상큼한 과일]","[디저트, 치즈]",화이트,"소비뇽 블랑 (Sauvignon Blanc) 85%, 게뷔르츠트라미너 (Gewurztraminer) 15%",5,1,3,1,소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 게뷔르츠트라미너 게뷔르츠트라미너,...,0,0,0,1,0,0,0,0,아주강한당도 아주약한산도 보통바디 아주약한타닌,"에밀리아나, 레이트 하베스트"
977,977,"[달콤한 과일, 빵, 상큼한 과일]",[치즈],화이트,"소비뇽 블랑 (Sauvignon Blanc) , 세미용 (Semillon)",1,4,3,1,소비뇽블랑 세미용 소비뇽블랑 세미용 소비뇽블랑 세미용 소비뇽블랑 세미용 소비뇽블랑 세미용,...,0,0,0,1,0,0,0,0,아주약한당도 강한산도 보통바디 아주약한타닌,"마리오니, 보르도 요크빌 하일랜드"
13342,13342,"[달콤한 과일, 디저트]","[디저트, 치즈]",화이트,리슬링 (Riesling) 100%,5,4,4,1,리슬링 리슬링 리슬링 리슬링 리슬링 리슬링 리슬링 리슬링 리슬링 리슬링,...,1,0,0,1,0,0,0,0,아주강한당도 강한산도 강한바디 아주약한타닌,"닥터 타니쉬, 베른카스텔 닥터 리슬링 아이스바인"
5732,5732,[],"[디저트, 치즈]",화이트,모스카토 비앙코 (Moscato Bianco) 100%,4,2,3,1,모스카토비앙코 모스카토비앙코 모스카토비앙코 모스카토비앙코 모스카토비앙코 모스카토비앙코 모스카토비앙코 모스카토비앙코 모스카토비앙코 모스카토비앙코,...,0,0,0,1,0,0,0,0,강한당도 약한산도 보통바디 아주약한타닌,보떼르 모스카토
12742,12742,"[달콤한 과일, 디저트, 상큼한 과일]",[치즈],화이트,"세미용 (Semillon) 80%, 소비뇽 블랑 (Sauvignon Blanc) 20%",5,4,4,1,세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용 소비뇽블랑 소비뇽블랑,...,1,0,0,1,0,0,0,0,아주강한당도 강한산도 강한바디 아주약한타닌,샤또 드 레인 비뇨
12684,12684,"[달콤한 과일, 기타]",[디저트],화이트,"세미용 (Semillon) 70%, 소비뇽 블랑 (Sauvignon Blanc) 20%, 뮈스까델 (Muscadelle) 10%",4,2,3,1,세미용 세미용 세미용 세미용 세미용 세미용 세미용 소비뇽블랑 소비뇽블랑 뮈스까델,...,0,0,0,1,0,0,0,0,강한당도 약한산도 보통바디 아주약한타닌,무쉬 시라노 꼬뜨 드 베르제락 무왈르
12690,12690,"[달콤한 과일, 디저트, 상큼한 과일]",[디저트],화이트,"세미용 (Semillon) 70%, 소비뇽 블랑 (Sauvignon Blanc) 20%, 뮈스까델 (Muscadelle) 10%",5,2,3,1,세미용 세미용 세미용 세미용 세미용 세미용 세미용 소비뇽블랑 소비뇽블랑 뮈스까델,...,0,0,0,1,0,0,0,0,아주강한당도 약한산도 보통바디 아주약한타닌,샤또 몽바지악
11144,11144,[],[디저트],화이트,세미용 (Semillon) 100%,5,2,3,1,세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용,...,0,0,0,1,0,0,0,0,아주강한당도 약한산도 보통바디 아주약한타닌,라스 따까스 세미용
11145,11145,[],[디저트],화이트,세미용 (Semillon) 100%,5,2,3,1,세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용 세미용,...,0,0,0,1,0,0,0,0,아주강한당도 약한산도 보통바디 아주약한타닌,둘세 세미용
12983,12983,"[달콤한 과일, 허브]","[디저트, 치즈]",화이트,소비뇽 블랑 (Sauvignon Blanc) 100%,1,5,4,1,소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑 소비뇽블랑,...,1,0,0,1,0,0,0,0,아주약한당도 아주강한산도 강한바디 아주약한타닌,"포레스트, 소비뇽 블랑"


#### 결론: aroma와 food가 비슷하므로 둘 중 하나만 사용하여 추천시스템 구현해도 괜찮은듯 최종적으로 맛: 0.3 , 종류: 0.2 , 음식:0.5 조합의 추천시스템 채택